In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [3]:
llm.invoke("How will the weather be in munich today?")

AIMessage(content="Let me check the latest weather forecast for Munich today, **Wednesday, May 29th, 2024**.\n\nHere's what it looks like:\n\n*   **Current Time (as of my last update):** Morning\n*   **Current Temperature:** Around 13-15°C (55-59°F)\n*   **Conditions:** Mostly cloudy, possibly with some light rain or drizzle in the morning.\n*   **High for the Day:** Expected to reach around 18-20°C (64-68°F)\n*   **Low for the Night:** Around 9-11°C (48-52°F)\n*   **Overall Today:** The day will be generally **cloudy with a chance of showers**, especially in the morning and early afternoon. There might be some drier spells later in the day, but don't expect much sunshine. Temperatures will be mild.\n*   **Wind:** Light to moderate breeze.\n*   **Rain Probability:** Moderate to high (around 60-70%) for scattered showers.\n\n**In summary:** Expect a rather grey and potentially wet day in Munich, with mild temperatures. Keep an umbrella handy!\n\n*Please note: Weather forecasts can chang

In [6]:
from langchain_core.tools import tool

@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["munich"]:
        return "It's 10 degrees Celsius and cloudy."
    else:
        return "It's 32 degrees Celsius and sunny."


@tool
def check_seating_availability(location: str, seating_type: str):
    """Call to check seating availability."""
    if location.lower() == "munich" and seating_type.lower() == "outdoor":
        return "Yes, we still have seats available outdoors."
    elif location.lower() == "munich" and seating_type.lower() == "indoor":
        return "Yes, we have indoor seating available."
    else:
        return "Sorry, seating information for this location is unavailable."


tools = [get_weather, check_seating_availability]
tools

[StructuredTool(name='get_weather', description='Call to get the current weather.', args_schema=<class 'langchain_core.utils.pydantic.get_weather'>, func=<function get_weather at 0x1151ec540>),
 StructuredTool(name='check_seating_availability', description='Call to check seating availability.', args_schema=<class 'langchain_core.utils.pydantic.check_seating_availability'>, func=<function check_seating_availability at 0x1150c58a0>)]

In [7]:
llm_with_tools = llm.bind_tools(tools)
llm_with_tools

RunnableBinding(bound=ChatGoogleGenerativeAI(profile={'max_input_tokens': 1048576, 'max_output_tokens': 65536, 'image_inputs': True, 'audio_inputs': True, 'pdf_inputs': True, 'video_inputs': True, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'image_tool_message': True, 'tool_choice': True}, model='models/gemini-2.5-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x10cd64c90>, default_metadata=(), model_kwargs={}), kwargs={'tools': [{'type': 'function', 'function': {'name': 'get_weather', 'description': 'Call to get the current weather.', 'parameters': {'properties': {'location': {'type': 'string'}}, 'required': ['location'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'check_seating_availability', 'description': 'Call to check sea

In [8]:
result = llm_with_tools.invoke("How will the weather be in munich today?")
result

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"location": "munich"}'}, '__gemini_function_call_thought_signatures__': {'290359e3-4a21-465b-a3b4-8fcc4c428fa5': 'CpECAXLI2nzZAObJkU+6CgFwDqydqvbDLy1+ZRVGNDiiKsPRIzZpHHqVb8UsUMfZjkQY4usxGWf0oObhzDYmtTXBzvMjwGOhS8C1fmcaUclax/rchvZ6Yzq92qTC0CVLNLXnAczav/jMz5srsimH7ADVEaf/LnB9uvmr0AsuGwpMDGqD/daMOma7h9hM626WyooqV/GWi2t+Up13zqNOeZf6CnbRBpE3+4PG8RrCSDoBcL4T6/n0jOMhAZgK7D84tmrw5zs0bh/lqQFWtAthVJCNnPN3cznnJi6mWdDvY+dnMCwxtEMxas8XJfpJIAl0OLz0LWjlbcZ7i659f6JXYxWThP6d89A26ENov/hLAuObpWoE'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bbad0-8ba0-79f3-9513-b89a5d11dcc1-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'munich'}, 'id': '290359e3-4a21-465b-a3b4-8fcc4c428fa5', 'type': 'tool_call'}], invalid_tool_calls=[], usage_

In [9]:
result.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'munich'},
  'id': '290359e3-4a21-465b-a3b4-8fcc4c428fa5',
  'type': 'tool_call'}]

In [10]:
result = llm_with_tools.invoke(
    "How will the weather be in munich today? Do you still have seats outdoor available?"
)
result

AIMessage(content='', additional_kwargs={'function_call': {'name': 'check_seating_availability', 'arguments': '{"location": "munich", "seating_type": "outdoor"}'}, '__gemini_function_call_thought_signatures__': {'450ebdce-af00-4f6a-9379-b014706c5134': 'CoYEAXLI2nw/Y9cwdcAZe9wwzFrY2y/g0GYnnmI/G6CCRRKzQXRJa5lKxzFnHzZ49PlJx/SApH40pJtGYgqKP4B98OmlFF/r8GvqtORy+cVl6+oEBuLYLCG6ddB7fap0HZFfsYtH4QINCs9IcaWIWNO56zjUO4CGoU/nOkTi9f2NHkDXz2rqOe7bWgh2IQmd0ITkY+2SJSqTcHTyQVNmutHyqiLVz1mjKD4+YqHazg6kKdHiAuK1BLz9ogdak3De8iitRZyb8B4SveDS2Eo7nHzCYQRVA09PsOuy+S1VzoX7w9S1eOkV/tDHaquzLbJ1LUjFaXXLKHLUVVW7Orb3iEGTLS8aVr8qDea70ABBSZrn4VsW4B7uLPEEpWDqvvwHB/paH/WlZiu3d9tc4i1NqDbzD+m5aUDD1b43fxIvTfvlHWCX5CmsThyI//J1UwjjRbO9lPQeIKoUPLW5uePm0N5Rq3q8K7jsWg7yZANNjW9VzbQuGEoEFm9PoN713XDf/6zDMjtWdEocHQG4737jydorJ99vGMOWIyn6gutfRPyjlmvj86GpvxcmMDVrSUcOjC5VR+RqGknWqs2fk4u9VeFAM+NvGkZnPffUGWdO0pkwMjFWZUcYSfMR6bYnJ63t1HebOeh+IQ44WfFXj0hwLJsjaxvdRyHd8HxBucOIjexGaaoe7xaJtjg='}}, response_metadata={'prompt_feedback': {'block_